**Importing Libraries**

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn

**Data Preprocessing**

In [ ]:
dataset = pd.read_csv('diabetes_prediction_dataset.csv')
# dataset

In [ ]:
X = dataset.iloc[:, 0:-1].values
Y = dataset.iloc[:, -1].values

In [ ]:
pd.DataFrame(X, columns = ["gender", "age", "hypertension","heart_disease","smoking_history","bmi","HbA1c_level","blood_glucose_level"])

In [ ]:
pd.DataFrame(Y, columns = ["diabetes"])

In [ ]:
# X

In [ ]:
# Y

**Data Encoding**

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 0] = le.fit_transform(X[:, 0])
# print(X)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [4])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
# print(X)

**Splitting Data into Training & Test Set**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

**Feature Scaling**

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

**ANN Construction**

In [ ]:
# ANN Initialization
ann = tf.keras.models.Sequential()

ann.add(tf.keras.layers.Dense(units=10, activation='elu'))       # Input Layer and 1st Hidden Layer (Activation Function: eLU)
ann.add(tf.keras.layers.Dense(units=10, activation='selu'))       # Second Hidden layer (AF: SeLU)
ann.add(tf.keras.layers.Dense(units=10, activation='gelu'))       # Second Hidden layer (AF: GeLU)
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))    # Output layer (AF: Sigmoid)


**ANN Training**

In [ ]:
# ANN Compilation (Optimizer: ADAMax, Loss Function: Binary Cross-Entropy)
ann.compile(optimizer = 'adamax', loss = 'binary_crossentropy', metrics = ['accuracy'])

# ANN Training (Batch Size: 32, Epochs: 100)
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

**Model Evaluation & Prediction Making**

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

**Confusion Matrix**

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
conf_matrix = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

# Model Performance Metrics
print("Model Accuracy:", accuracy_score(y_test, y_pred))
print("Model Specificity:", tn / (tn+fp))
print("Model Specificity:", tp / (tp+fn))

In [ ]:
fig, ax = plt.subplots(figsize=(7.5, 7.5))
ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')
 
plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix', fontsize=18)
plt.show()